[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/DeZero-Koki/blob/master/from_scratch_3/step18.ipynb)

# 제1고지 미분 자동 계산

In [1]:
import numpy as np


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


In [2]:
# class Exp(Function):
#     def forward(self, x):
#         return np.exp(x)

#     def backward(self, gy):
#         x = self.input.data
#         gx = np.exp(x) * gy
#         return gx


# def exp(x):
#     return Exp()(x)


In [3]:
def numerical_diff(f, x, eps=1e-4):
    """

      param:
        f (Function): 미분의 대상이 되는 함수
        x (Variable): 미분을 계산하는 변수
        eps (float): 작은 값
    """
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

# 제2고지 자연스러운 코드로

## 11단계 가변 길이 인수(순전파 편)

## 12단계 가변 길이 인수(개선 편)

## 13단계 가변 길이 인수(역전파 편)

## 14단계 같은 변수 반복 사용

## 15단계 복잡한 계산 그래프(이론 편)

## 16단계 복잡한 계산 그래프(구현 편)

## 17단계 메모리 관리와 순환 참조

In [4]:
import weakref


class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, *inputs):
        xs = [ x.data for x in inputs ]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple): ys = ys,
        outputs = [ Variable(as_array(y)) for y in ys ]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]  # 변경!
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [5]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [ output().grad for output in f.outputs ]  # 변경!
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

    def cleargrad(self):
        self.grad = None


## 18단계 메모리 절약 모드

### 18.1 필요 없는 미분값 삭제

In [6]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


In [7]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

1.0 1.0
2.0 1.0


In [8]:
import numpy as np
import weakref


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(
        self,
        retain_grad=False  # 추가!
    ):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [ output().grad for output in f.outputs ]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            ########## 추가 ##########
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y는 약한 참조(weakref)
            #########################

    def cleargrad(self):
        self.grad = None


In [9]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


### 18.2 Function 클래스 복습

In [10]:
import weakref


class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, *inputs):
        xs = [ x.data for x in inputs ]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple): ys = ys,
        outputs = [ Variable(as_array(y)) for y in ys ]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs  # 결과값을 보관하는 부분
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 18.3 Config 클래스를 활용한 모드 전환

In [11]:
class Config:
    enable_backprop = True

In [12]:
import weakref


class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, *inputs):
        xs = [ x.data for x in inputs ]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple): ys = ys,
        outputs = [ Variable(as_array(y)) for y in ys ]

        if Config.enable_backprop:  # 추가!
            self.generation = max([x.generation for x in inputs])  # ① 세대 설정
            for output in outputs:
                output.set_creator(self)  # ② 연결 설정
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 18.4 모드 전환

In [13]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


In [14]:
import gc; gc.collect()
import psutil


m0 = psutil.virtual_memory()._asdict()["free"]

Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()

m1 = psutil.virtual_memory()._asdict()["free"]
print(f"{(m0-m1)/1024:.2f}KB")
del x, y; gc.collect()


m0 = psutil.virtual_memory()._asdict()["free"]

Config.enable_backprop = False
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))

m1 = psutil.virtual_memory()._asdict()["free"]
print(f"{(m0-m1)/1024:.2f}KB")
del x, y; gc.collect()

51452.00KB
11468.00KB


0

### 18.5 with 문을 활용한 모드 전환

In [15]:
# f = open("sample.txt", 'w')
# f.write("hello world!")
# f.close()

# with open("sample.txt", 'w') as f:
#     f.write("hello world!")

In [16]:
# with using_config("enable_backprop", False):
#     x = Variable(np.array(2.0))
#     y = square(x)

In [17]:
import contextlib

@contextlib.contextmanager
def config_test():
    print("start")  # 전처리
    try:
        yield
    finally:
        print("done")  # 후처리

with config_test():
    print("process...")

start
process...
done


In [18]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [19]:
with using_config("enable_backprop", False):
    x = Variable(np.array(2.0))
    y = square(x)

In [20]:
def no_grad():
    return using_config("enable_backprop", False)

In [21]:
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)